In [37]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import sys
from scipy.integrate import odeint
from scipy.integrate import solve_ivp

In [38]:
#vedio
def function1(time,dev,data):

# wourld wide average carbon generation by unit power
    # cite: Obringer, Renee, et al. "The overlooked environmental footprint of increasing Internet use." Resources, Conservation and Recycling 167 (2021): 105389.

    p1=np.array([820,702,490,12,24,230,11,12,48,38,27])   #g Co2/kW
    p2=np.array([0.38106531, 0.03041596, 0.23030303, 0.10231508, 0.1582157 ,
           0.02394128, 0.04509693, 0.00252212, 0.02228496, 0.00338792,
           0.00045172])
    p_c=np.sum(p1*p2)


    #client - Desktop
    if dev=='notebook':

        p_client= 30 # W
    elif dev=='desktop':
    #client - Desktop
        p_client=150 #W
    elif dev=='phone':
    # phone
        p_client=10 # W
    

    # home network  bandwidth

    b_net=16 # GB/hour

    #total_gCO2e = data_utilsation*(data_centre_electricity usage +transmission_electricity usage)*gCO2e_for_electricity_generation

    # electricity consumption during operation , three parts:servers, clients, routers, per hour

    es_ser=  0.01*p_c     #  kW/GB*g/kW=g/GB
    es_cli=  p_client*1/1000/(b_net)*p_c        #kW/h*1hour/GB in one hour * g/GB =g/GB
    es_rou=  0.06*p_c    # kW/GB*g/kW=g/GB

    # energy required for production, three parts: servers, clients, routers
    
    # google total 2500,000 servers, Youtube use 6-7 out of 34 data centers of google .  
    # cite: https://www.geeksforgeeks.org/how-does-youtube-store-and-analyze-such-huge-amount-of-data/
    # cite: https://en.wikipedia.org/wiki/Google_data_centers
    # cite: https://corporate.delltechnologies.com/en-gb/social-impact/advancing-sustainability/sustainable-products-and-services/product-carbon-footprints.htm#tab0=3
    # server life time : 4 years  
    # assume router has the same footprint as server.
    # one server foot print in manufacture and transport.
    
    # for youtube server
    p_single_server_list=[25.1/100*4800, 8.3/100*8160 ,7490*16.8/100 ,26.9*6180/100, 8970*20.4/100]
    p_single_server=np.mean(p_single_server_list)
    N_servers=2500000*(7/34)
    
    # per user per year
    #cite https://blog.hootsuite.com/youtube-stats-marketers/
    
    N_user=2e9 
    
    # for personal computer desktop
    # cite: https://corporate.delltechnologies.com/en-gb/social-impact/advancing-sustainability/sustainable-products-and-services/product-carbon-footprints.htm#tab0=1
    pp1=np.array([512, 214, 488, 485,491,388,575,501,583,210,579,451,2496,1180,765,642,537,346])
    pp2=np.array([28.5, 26.3, 42.1,49,51,33,44,51,50,27,44,44,73.8,61.7,55.4,46.2,34.4,33.8])
    p_single_pc_list=(pp1*(100-pp2)/100)
    p_single_pc=np.mean(p_single_pc_list)
    
    
    # for personal computer laptop
    # cite: https://corporate.delltechnologies.com/en-gb/social-impact/advancing-sustainability/sustainable-products-and-services/product-carbon-footprints.htm#tab0=0
    pp1=np.array([361,364,241,329,297,305,249,271,315,235,302,315,318,231,533,304,509,380,286,322,273,355])
    pp2=np.array([9.7,13,12.6,10.8,5.3,9.1,19.8,22,14.6,17.2,14.6,15.3,15,13,8.9,16.2,9.9,15.3,20,13.9,10,8.3])
    p_single_nb_list=(pp1*(100-pp2)/100)
    p_single_nb=np.mean(p_single_nb_list)
    
    # for cell phone
    # https://www.apple.com/environment/pdf/products/iphone/iPhone_12_PER_Oct2020.pdf
    # https://reboxed.co/blogs/outsidethebox/the-carbon-footprint-of-your-phone-and-how-you-can-reduce-it
    # use iphone as a example


    p_single_phone_list=np.array([86, 82, 70, 64, 57, 86, 80, 72, 77, 70, 62, 79, 68, 57, 67, 56])  # kg co2 in life time
    p_single_phone=np.mean(p_single_phone_list)
    

    ep_ser=N_servers*p_single_server*(b_net/16)*1000/N_user
    ep_cli=p_single_phone*1000   #g co2
    ep_rou=ep_ser  # same as server

    lifetime1=4*365*24 # year
    lifetime2=4*365*24 # year
    lifetime3=4*365*24 # year

    # total energy consumption is the sum of the above two parts.


    E1=es_ser+es_cli+es_rou
    E2=ep_ser/lifetime1+ep_cli/lifetime2+ep_rou/lifetime3

    E=data*E1+T*E2


    return E

In [39]:
def function2(time,dev,data):  # chat
    # cite: Obringer, Renee, et al. "The overlooked environmental footprint of increasing Internet use." Resources, Conservation and Recycling 167 (2021): 105389.
    p1=np.array([820,702,490,12,24,230,11,12,48,38,27])   #g Co2/kW
    p2=np.array([0.38106531, 0.03041596, 0.23030303, 0.10231508, 0.1582157 ,
           0.02394128, 0.04509693, 0.00252212, 0.02228496, 0.00338792,
           0.00045172])
    p_c=np.sum(p1*p2)


    #client - Desktop
    if dev=='notebook':

        p_client= 30 # W
    elif dev=='desktop':
    #client - Desktop
        p_client=150 #W
    elif dev=='phone':
    # phone
        p_client=10 # W

    # home network  bandwidth

    b_net=0.5 # GB/hour

    #total_gCO2e = data_utilsation*(data_centre_electricity usage +transmission_electricity usage)*gCO2e_for_electricity_generation

    # electricity consumption during operation , three parts:servers, clients, routers, per hour

    es_ser=  0.01*p_c     #  kW/GB*g/kW=g/GB
    es_cli=  p_client*1/1000/(b_net)*p_c        #kW/h*1hour/GB in one hour * g/GB =g/GB
    es_rou=  0.06*p_c    # kW/GB*g/kW=g/GB

    # energy required for production, three parts: servers, clients, routers

    # google total 2500,000 servers, Youtube use 6-7 out of 34 data centers of google .  
    # cite: https://www.geeksforgeeks.org/how-does-youtube-store-and-analyze-such-huge-amount-of-data/
    # cite: https://en.wikipedia.org/wiki/Google_data_centers
    # cite: https://corporate.delltechnologies.com/en-gb/social-impact/advancing-sustainability/sustainable-products-and-services/product-carbon-footprints.htm#tab0=3
    # server life time : 4 years  
    # assume router has the same footprint as server.
    # one server foot print in manufacture and transport.
    
    # for youtube server
    p_single_server_list=[25.1/100*4800, 8.3/100*8160 ,7490*16.8/100 ,26.9*6180/100, 8970*20.4/100]
    p_single_server=np.mean(p_single_server_list)
    N_servers=2500000*(7/34)
    
    # per user per year
    #cite https://blog.hootsuite.com/youtube-stats-marketers/
    
    N_user=2e9 
    
    # for personal computer desktop
    # cite: https://corporate.delltechnologies.com/en-gb/social-impact/advancing-sustainability/sustainable-products-and-services/product-carbon-footprints.htm#tab0=1
    pp1=np.array([512, 214, 488, 485,491,388,575,501,583,210,579,451,2496,1180,765,642,537,346])
    pp2=np.array([28.5, 26.3, 42.1,49,51,33,44,51,50,27,44,44,73.8,61.7,55.4,46.2,34.4,33.8])
    p_single_pc_list=(pp1*(100-pp2)/100)
    p_single_pc=np.mean(p_single_pc_list)
    
    
    # for personal computer laptop
    # cite: https://corporate.delltechnologies.com/en-gb/social-impact/advancing-sustainability/sustainable-products-and-services/product-carbon-footprints.htm#tab0=0
    pp1=np.array([361,364,241,329,297,305,249,271,315,235,302,315,318,231,533,304,509,380,286,322,273,355])
    pp2=np.array([9.7,13,12.6,10.8,5.3,9.1,19.8,22,14.6,17.2,14.6,15.3,15,13,8.9,16.2,9.9,15.3,20,13.9,10,8.3])
    p_single_nb_list=(pp1*(100-pp2)/100)
    p_single_nb=np.mean(p_single_nb_list)
    
    # for cell phone
    # https://www.apple.com/environment/pdf/products/iphone/iPhone_12_PER_Oct2020.pdf
    # https://reboxed.co/blogs/outsidethebox/the-carbon-footprint-of-your-phone-and-how-you-can-reduce-it
    # use iphone as a example


    p_single_phone_list=np.array([86, 82, 70, 64, 57, 86, 80, 72, 77, 70, 62, 79, 68, 57, 67, 56])  # kg co2 in life time
    p_single_phone=np.mean(p_single_phone_list)
    

    ep_ser=N_servers*p_single_server*(b_net/16)*1000/N_user
    ep_cli=p_single_phone*1000   #g co2
    ep_rou=ep_ser  # same as server

    lifetime1=4*365*24 # year
    lifetime2=4*365*24 # year
    lifetime3=4*365*24 # year

    # total energy consumption is the sum of the above two parts.


    E1=es_ser+es_cli+es_rou
    E2=ep_ser/lifetime1+ep_cli/lifetime2+ep_rou/lifetime3

    E=data*E1+T*E2


    return E

In [40]:
def function3(time,dev,data): # web
    # cite: Obringer, Renee, et al. "The overlooked environmental footprint of increasing Internet use." Resources, Conservation and Recycling 167 (2021): 105389.
    p1=np.array([820,702,490,12,24,230,11,12,48,38,27])   #g Co2/kW
    p2=np.array([0.38106531, 0.03041596, 0.23030303, 0.10231508, 0.1582157 ,
           0.02394128, 0.04509693, 0.00252212, 0.02228496, 0.00338792,
           0.00045172])
    p_c=np.sum(p1*p2)


    #client - Desktop
    if dev=='notebook':

        p_client= 30 # W
    elif dev=='desktop':
    #client - Desktop
        p_client=150 #W
    elif dev=='phone':
    # phone
        p_client=10 # W

    # home network  bandwidth

    b_net=0.4 # GB/hour

    #total_gCO2e = data_utilsation*(data_centre_electricity usage +transmission_electricity usage)*gCO2e_for_electricity_generation

    # electricity consumption during operation , three parts:servers, clients, routers, per hour

    es_ser=  0.01*p_c     #  kW/GB*g/kW=g/GB
    es_cli=  p_client*1/1000/(b_net)*p_c        #kW/h*1hour/GB in one hour * g/GB =g/GB
    es_rou=  0.06*p_c    # kW/GB*g/kW=g/GB

    # energy required for production, three parts: servers, clients, routers

    # google total 2500,000 servers, Youtube use 6-7 out of 34 data centers of google .  
    # cite: https://www.geeksforgeeks.org/how-does-youtube-store-and-analyze-such-huge-amount-of-data/
    # cite: https://en.wikipedia.org/wiki/Google_data_centers
    # cite: https://corporate.delltechnologies.com/en-gb/social-impact/advancing-sustainability/sustainable-products-and-services/product-carbon-footprints.htm#tab0=3
    # server life time : 4 years  
    # assume router has the same footprint as server.
    # one server foot print in manufacture and transport.
    
    # for youtube server
    p_single_server_list=[25.1/100*4800, 8.3/100*8160 ,7490*16.8/100 ,26.9*6180/100, 8970*20.4/100]
    p_single_server=np.mean(p_single_server_list)
    N_servers=2500000*(7/34)
        
    # per user per year
    #cite https://blog.hootsuite.com/youtube-stats-marketers/
    
    N_user=2e9 
    
    # for personal computer desktop
    # cite: https://corporate.delltechnologies.com/en-gb/social-impact/advancing-sustainability/sustainable-products-and-services/product-carbon-footprints.htm#tab0=1
    pp1=np.array([512, 214, 488, 485,491,388,575,501,583,210,579,451,2496,1180,765,642,537,346])
    pp2=np.array([28.5, 26.3, 42.1,49,51,33,44,51,50,27,44,44,73.8,61.7,55.4,46.2,34.4,33.8])
    p_single_pc_list=(pp1*(100-pp2)/100)
    p_single_pc=np.mean(p_single_pc_list)
    
    
    # for personal computer laptop
    # cite: https://corporate.delltechnologies.com/en-gb/social-impact/advancing-sustainability/sustainable-products-and-services/product-carbon-footprints.htm#tab0=0
    pp1=np.array([361,364,241,329,297,305,249,271,315,235,302,315,318,231,533,304,509,380,286,322,273,355])
    pp2=np.array([9.7,13,12.6,10.8,5.3,9.1,19.8,22,14.6,17.2,14.6,15.3,15,13,8.9,16.2,9.9,15.3,20,13.9,10,8.3])
    p_single_nb_list=(pp1*(100-pp2)/100)
    p_single_nb=np.mean(p_single_nb_list)
    
    # for cell phone
    # https://www.apple.com/environment/pdf/products/iphone/iPhone_12_PER_Oct2020.pdf
    # https://reboxed.co/blogs/outsidethebox/the-carbon-footprint-of-your-phone-and-how-you-can-reduce-it
    # use iphone as a example


    p_single_phone_list=np.array([86, 82, 70, 64, 57, 86, 80, 72, 77, 70, 62, 79, 68, 57, 67, 56])  # kg co2 in life time
    p_single_phone=np.mean(p_single_phone_list)
    

    ep_ser=N_servers*p_single_server*(b_net/16)*1000/N_user
    ep_cli=p_single_phone*1000   #g co2
    ep_rou=ep_ser  # same as server

    lifetime1=4*365*24 # year
    lifetime2=4*365*24 # year
    lifetime3=4*365*24 # year

    # total energy consumption is the sum of the above two parts.


    E1=es_ser+es_cli+es_rou
    E2=ep_ser/lifetime1+ep_cli/lifetime2+ep_rou/lifetime3

    E=data*E1+T*E2


    return E

In [41]:
def function4(time,dev,data): #game
    # cite: Obringer, Renee, et al. "The overlooked environmental footprint of increasing Internet use." Resources, Conservation and Recycling 167 (2021): 105389.
    p1=np.array([820,702,490,12,24,230,11,12,48,38,27])   #g Co2/kW
    p2=np.array([0.38106531, 0.03041596, 0.23030303, 0.10231508, 0.1582157 ,
           0.02394128, 0.04509693, 0.00252212, 0.02228496, 0.00338792,
           0.00045172])
    p_c=np.sum(p1*p2)


    #client - Desktop
    if dev=='notebook':

        p_client= 30 # W
    elif dev=='desktop':
    #client - Desktop
        p_client=500 #W
    elif dev=='phone':
    # phone
        p_client=10 # W

    # home network  bandwidth

    b_net=0.3 # GB/hour

    #total_gCO2e = data_utilsation*(data_centre_electricity usage +transmission_electricity usage)*gCO2e_for_electricity_generation

    # electricity consumption during operation , three parts:servers, clients, routers, per hour

    es_ser=  0.01*p_c     #  kW/GB*g/kW=g/GB
    es_cli=  p_client*1/1000/(b_net)*p_c        #kW/h*1hour/GB in one hour * g/GB =g/GB
    es_rou=  0.06*p_c    # kW/GB*g/kW=g/GB

    # energy required for production, three parts: servers, clients, routers
    # google total 2500,000 servers, Youtube use 6-7 out of 34 data centers of google .  
    # cite: https://www.geeksforgeeks.org/how-does-youtube-store-and-analyze-such-huge-amount-of-data/
    # cite: https://en.wikipedia.org/wiki/Google_data_centers
    # cite: https://corporate.delltechnologies.com/en-gb/social-impact/advancing-sustainability/sustainable-products-and-services/product-carbon-footprints.htm#tab0=3
    # server life time : 4 years  
    # assume router has the same footprint as server.
    # one server foot print in manufacture and transport.
    
    # for youtube server
    p_single_server_list=[25.1/100*4800, 8.3/100*8160 ,7490*16.8/100 ,26.9*6180/100, 8970*20.4/100]
    p_single_server=np.mean(p_single_server_list)
    N_servers=2500000*(7/34)
    
        
    # per user per year
    #cite https://blog.hootsuite.com/youtube-stats-marketers/
    
    N_user=2e9 
    
    # for personal computer desktop
    # cite: https://corporate.delltechnologies.com/en-gb/social-impact/advancing-sustainability/sustainable-products-and-services/product-carbon-footprints.htm#tab0=1
    pp1=np.array([512, 214, 488, 485,491,388,575,501,583,210,579,451,2496,1180,765,642,537,346])
    pp2=np.array([28.5, 26.3, 42.1,49,51,33,44,51,50,27,44,44,73.8,61.7,55.4,46.2,34.4,33.8])
    p_single_pc_list=(pp1*(100-pp2)/100)
    p_single_pc=np.mean(p_single_pc_list)
    
    
    # for personal computer laptop
    # cite: https://corporate.delltechnologies.com/en-gb/social-impact/advancing-sustainability/sustainable-products-and-services/product-carbon-footprints.htm#tab0=0
    pp1=np.array([361,364,241,329,297,305,249,271,315,235,302,315,318,231,533,304,509,380,286,322,273,355])
    pp2=np.array([9.7,13,12.6,10.8,5.3,9.1,19.8,22,14.6,17.2,14.6,15.3,15,13,8.9,16.2,9.9,15.3,20,13.9,10,8.3])
    p_single_nb_list=(pp1*(100-pp2)/100)
    p_single_nb=np.mean(p_single_nb_list)
    
    # for cell phone
    # https://www.apple.com/environment/pdf/products/iphone/iPhone_12_PER_Oct2020.pdf
    # https://reboxed.co/blogs/outsidethebox/the-carbon-footprint-of-your-phone-and-how-you-can-reduce-it
    # use iphone as a example


    p_single_phone_list=np.array([86, 82, 70, 64, 57, 86, 80, 72, 77, 70, 62, 79, 68, 57, 67, 56])  # kg co2 in life time
    p_single_phone=np.mean(p_single_phone_list)
    

    ep_ser=N_servers*p_single_server*(b_net/16)*1000/N_user
    ep_cli=p_single_phone*1000   #g co2
    ep_rou=ep_ser  # same as server

    lifetime1=4*365*24 # year
    lifetime2=4*365*24 # year
    lifetime3=4*365*24 # year

    # total energy consumption is the sum of the above two parts.


    E1=es_ser+es_cli+es_rou
    E2=ep_ser/lifetime1+ep_cli/lifetime2+ep_rou/lifetime3

    E=data*E1+T*E2


    return E

In [42]:
# please input your app as four categrates
app='video'
app='chat'
app='game'
app='web'
# how long you use your device per day in hour?
T=12  # hours
# what kind of device you use
Dev='laptop'
Dev='phone'
Dev='desktop'
# how many data you use?
Data= 10  #GB

w_carbon1=function1(T,Dev,Data)
w_carbon2=function2(T,Dev,Data)
w_carbon3=function3(T,Dev,Data)
w_carbon4=function4(T,Dev,Data)

In [43]:
w_carbon1

388.7702476671178